In [1]:
import pandas as pd
import nltk.corpus
import random
import os
from collections import defaultdict, Counter
from tqdm import tqdm

# See where enwiki/editor_talk_scores.csv was extracted from

In [2]:
data = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores.csv')
data

,article,thread_title,editor,editor_talk,other_talk,#editor_turns,#other_turns,editor_score
0,"""Tiger"" William Dunlop",Current image,Deconstructhis,I've noticed that the current image that's acc...,I removed the deletion tag because the image i...,2,3,0.033058
1,"""Tiger"" William Dunlop",Current image,Skinsmoke,::Good move ukexpat. That was really helpful (...,I've noticed that the current image that's acc...,1,4,0.482759
2,"""Tiger"" William Dunlop",Current image,Ukexpat,I removed the deletion tag because the image i...,I've noticed that the current image that's acc...,1,4,0.392157
3,"""Tiger"" William Dunlop",Requested move,CJ3370,"William ""Tiger"" Dunlop → William ""Tiger"" Dunlo...",The following discussion is an archived discus...,2,6,0.826415
4,"""Tiger"" William Dunlop",Requested move,CJ3370,"William ""Tiger"" Dunlop → William ""Tiger"" Dunlo...",The following discussion is an archived discus...,2,6,0.826415
5,"""Tiger"" William Dunlop",Requested move,Deconstructhis,Not a problem to merge if that is the most app...,The following discussion is an archived discus...,1,7,0.033058
6,"""Tiger"" William Dunlop",Requested move,Deconstructhis,Not a problem to merge if that is the most app...,The following discussion is an archived discus...,1,7,0.033058
7,"""Tiger"" William Dunlop",Requested move,Skinsmoke,User:CJ3370/William_Tiger_Dunlop → William Tig...,The following discussion is an archived discus...,3,5,0.456897
8,"""Tiger"" William Dunlop",Requested move,Skinsmoke,User:CJ3370/William_Tiger_Dunlop → William Tig...,The following discussion is an archived discus...,3,5,0.456897
9,"""V"" Is for Vagina","Why is the ""i"" capitalized?",Gunmetal Angel,"""I"" is supposed to be capatalized in ""is"" in t...",It doesn't make sense and it doesn't represent...,1,9,1.000000


In [3]:
len(data['article'].unique())

10379

In [4]:
talk2 = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/talk_filtered_2.csv')
talk3 = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/talk_filtered_3.csv')

talk2.columns

Index(['contribution_id', 'article', 'thread', 'timestamp', 'username',
       'thread_type', 'text'],
      dtype='object')

In [5]:
print(len(talk2['article'].unique()))
print(len(talk3['article'].unique()))

11217
6006


# See how many additions and deletions happen repeatedly by same author (Carolyn's editor score problem)

In [35]:
# Get subset of talk pagenames
with open('/home/michael/school/research/wp/talk_pagenames_100k_filtered2.txt') as f:
    pagenames = f.read().splitlines()
    
pagenames_sample = random.sample(pagenames, 100)
pagenames_sample

['Lexx',
 'Hikitsu',
 'Great Ormond Street Hospital',
 'M198 howitzer',
 'Avicennism',
 'Data driven science',
 'Seventh Studio Album (Missy Elliott album)',
 'Mechitza',
 'Fursona',
 "Alliance of Girls' Schools Australasia",
 'University of Western Sydney',
 'Wolfgang Amadeus Phoenix',
 'Zhang Zhixin',
 'Media franchise',
 'USS Cumberland (1842)',
 'Riverline (Hobart)',
 'Paul Kantner',
 'Irvine, California',
 'Saez',
 'Agave attenuata',
 'Rachel Barrett',
 'HFSX',
 'Mitochondrial encephalomyopathy',
 'Body piercing aftercare',
 'Antony Hamilton',
 'Emergency service',
 'Financial Industry Regulatory Authority',
 'Piotr Iljicz Czajkowskij',
 'Ben Bova',
 'William III of the Netherlands',
 'Battle of Quebec (ice hockey)',
 'Popples',
 'September 26',
 'List of Continuum episodes',
 'Cardiff International Sports Stadium',
 'State Correctional Institution – Greene',
 'MAP2K7',
 '2006 World Series of Poker results',
 'Initial Reaction Force',
 'Lozen',
 'List of business schools in Europe

In [4]:
# Make pagenames sample correct formatting
pagenames_formatted = ["{}_diff.csv".format(name.lower().replace(' ', '_').replace('/', '_')) for name in pagenames_sample]
pagenames_formatted

['.nes_diff.csv',
 'world_club_challenge_diff.csv',
 'étienne_geoffroy_saint-hilaire_diff.csv',
 'melanosis_bulborum_and_aberrant_dermal_melanocytosis_diff.csv',
 'be_here_now_(book)_diff.csv',
 'china_diff.csv',
 'turkmen_people_diff.csv',
 'list_of_tallest_buildings_in_knoxville_diff.csv',
 '8"_floppy_disk_diff.csv',
 'obliteration_by_incorporation_diff.csv',
 'nba_all-star_game_most_valuable_player_award_diff.csv',
 'mandela,_massachusetts_diff.csv',
 'kambarasam_diff.csv',
 'max_hastings_diff.csv',
 'potassium_hypomanganate_diff.csv',
 'clayton_wilbury_diff.csv',
 "alliance_of_girls'_schools_australasia_diff.csv",
 'babies_in_black_diff.csv',
 'daniş_tunalıgil_diff.csv',
 'oklahoma_state_university_–_okmulgee_diff.csv',
 'eugene_mallove_diff.csv',
 'history_of_athletics_at_stone_bridge_high_school_diff.csv',
 'ralph_neville,_1st_baron_neville_de_raby_diff.csv',
 'gerrards_cross_tunnel_diff.csv',
 'fluorescent_adolescent_diff.csv',
 '613_commandments_diff.csv',
 'loanword_diff.csv',

In [5]:
with open('/home/michael/school/research/wp/talk_pagenames_diff_sample.txt', 'w') as out:
    for p in pagenames_formatted:
        out.write(p + '\n')

In [28]:
# DIFFS: Write command as shell script
with open('/home/michael/school/research/wp/copy_diffs.sh', 'w') as out:
    out.write("#! /bin/bash\n")
    for p in pagenames_formatted:
        p_new = p.replace("(", "\(").replace(")", "\)").replace("&", "\&")
        cmd = "scp erebor:/usr2/mamille2/enwiki_diffs/\"{0}\" /home/michael/school/research/wp/enwiki_diffs/\n".format(p_new)
        out.write(cmd)

## Copy edit scores

In [48]:
# Make pagenames sample correct formatting
# pagenames_formatted = ["{:s}_edit_scores.csv".format(name.lower().replace(' ', '_').replace('/', '_').replace("(", "\(").replace(")", "\)").replace("&", "\&").replace('"', '\"')) for name in pagenames_sample]
pagenames_formatted = ["{:s}_edit_scores.csv".format(name.lower().replace(' ', '_').replace('/', '_').replace("&", "\&").replace('"', '\"')) for name in pagenames_sample]
pagenames_formatted

['lexx_edit_scores.csv',
 'hikitsu_edit_scores.csv',
 'great_ormond_street_hospital_edit_scores.csv',
 'm198_howitzer_edit_scores.csv',
 'avicennism_edit_scores.csv',
 'data_driven_science_edit_scores.csv',
 'seventh_studio_album_(missy_elliott_album)_edit_scores.csv',
 'mechitza_edit_scores.csv',
 'fursona_edit_scores.csv',
 "alliance_of_girls'_schools_australasia_edit_scores.csv",
 'university_of_western_sydney_edit_scores.csv',
 'wolfgang_amadeus_phoenix_edit_scores.csv',
 'zhang_zhixin_edit_scores.csv',
 'media_franchise_edit_scores.csv',
 'uss_cumberland_(1842)_edit_scores.csv',
 'riverline_(hobart)_edit_scores.csv',
 'paul_kantner_edit_scores.csv',
 'irvine,_california_edit_scores.csv',
 'saez_edit_scores.csv',
 'agave_attenuata_edit_scores.csv',
 'rachel_barrett_edit_scores.csv',
 'hfsx_edit_scores.csv',
 'mitochondrial_encephalomyopathy_edit_scores.csv',
 'body_piercing_aftercare_edit_scores.csv',
 'antony_hamilton_edit_scores.csv',
 'emergency_service_edit_scores.csv',
 'finan

In [49]:
# EDIT_SCORES: Write command as shell script
with open('/home/michael/school/research/wp/copy_edit_scores.sh', 'w') as out:
    out.write("#! /bin/bash\n")
    for p in pagenames_formatted:
        p_new = p.replace("(", "\(").replace(")", "\)")
        cmd = "scp erebor:/usr2/mamille2/enwiki_edit_scores/\"{0}\" /home/michael/school/research/wp/enwiki_edit_scores/\n".format(p_new)
        out.write(cmd)

## Use editor scores

In [2]:
def is_stop(wd):
    return wd in nltk.corpus.stopwords.words('english')

In [6]:
# Load editor scores
dirpath = '/home/michael/school/research/wp/enwiki_edit_scores/'
double_count_percentage = []

for fname in tqdm(sorted(os.listdir(dirpath))):
    tqdm.write("File: {}".format(fname))
    fpath = os.path.join(dirpath, fname)
    edit_scores = pd.read_csv(fpath, parse_dates=['edit_timestamp'])
    
    # Check how many tokens added, then removed, then added back again or vv
    total_toks_edited = []
    total_double_counted = []

    # ed_wds = {'adds': defaultdict(Counter()), 'dels': defauldict(Counter())} # editor: text added/deleted
    orig_edit = {'adds': Counter(), 'dels': Counter()}
    this_edit = {'adds': Counter(), 'dels': Counter()}

    threads = sorted(set(edit_scores['thread_title']))

#     for t in tqdm(threads, desc='threads'):
    for t in threads:
        tqdm.write("Thread: {}".format(t))
        rows = edit_scores[edit_scores['thread_title']==t]

        toks_edited = 0
        toks_double_counted = 0

        for r in rows.itertuples(): # Check starting at ea row
            potentials = {'adds': Counter(), 'dels': Counter()} # potential to be removed and replaced or vv
            double_counted = {'adds': Counter(), 'dels': Counter()} # put in, removed, put back in

            orig_ed = r.editor
            orig_ts = r.edit_timestamp

            if isinstance(r.additions, str):
                orig_edit['adds'] = Counter([w for w in r.additions.split() if not is_stop(w)])
            else: 
                orig_edit['adds'] = Counter()

            if isinstance(r.deletions, str):
                orig_edit['dels'] = Counter([w for w in r.deletions.split() if not is_stop(w)])
            else: 
                orig_edit['dels'] = Counter()

            toks_edited += sum(orig_edit['adds'].values()) + sum(orig_edit['dels'].values())

            rest_rows = rows[rows['edit_timestamp'] > orig_ts]

            for other_r in rest_rows.itertuples():
                if other_r.editor != orig_ed: 

                    # Check for intersection of additions w deletions, vv
                    if isinstance(other_r.additions, str):
                        this_edit['adds'] = Counter([w for w in other_r.additions.split() if not is_stop(w)])
                    else: 
                        this_edit['adds'] = Counter()
                    if isinstance(other_r.deletions, str):
                        this_edit['dels'] = Counter([w for w in other_r.deletions.split() if not is_stop(w)])
                    else: 
                        this_edit['dels'] = Counter()

                    # Additions subsequently deleted
                    unique_adds = orig_edit['adds'] - this_edit['dels']
                    intersection = orig_edit['adds'] - unique_adds # Will be nothing if no overlap
                    potentials['adds'] += intersection

                    # Deletions subsequently added
                    unique_dels = orig_edit['dels'] - this_edit['adds']
                    intersection = orig_edit['dels'] - unique_dels # Will be nothing if no overlap
                    potentials['dels'] += intersection

                else: # Check potentials to see if address them

                    if isinstance(other_r.additions, str):
                        this_edit['adds'] = Counter([w for w in other_r.additions.split() if not is_stop(w)])
                    else: 
                        this_edit['adds'] = Counter()
                    if isinstance(other_r.deletions, str):
                        this_edit['dels'] = Counter([w for w in other_r.deletions.split() if not is_stop(w)])
                    else: 
                        this_edit['dels'] = Counter()

                    # Additions subsequently deleted
                    unique_adds = potentials['adds'] - this_edit['dels']
                    intersection = potentials['adds'] - unique_adds # Will be nothing if no overlap
                    double_counted['adds'] += intersection

                    # Deletions subsequently added
                    unique_dels = potentials['dels'] - this_edit['adds']
                    intersection = potentials['dels'] - unique_dels # Will be nothing if no overlap
                    double_counted['dels'] += intersection

            if len(double_counted['adds'].values()) > 0 or len(double_counted['dels'].values()) > 0:
                print(double_counted)

            toks_double_counted += sum(double_counted['adds'].values()) + sum(double_counted['dels'].values())

        tqdm.write('{}/{} tokens changed back'.format(toks_double_counted, toks_edited))
        total_toks_edited.append(toks_edited)
        total_double_counted.append(toks_double_counted)
        
    double_count_percentage.append(sum(total_double_counted)/sum(total_toks_edited))

print(sum(double_count_percentage)/len(double_count_percentage))


               | 0/96 [00:00<?, ?it/s]
 23%|██▎       | 22/96 [36:41<2:03:25, 100.07s/it]
               | 0/96 [00:00<?, ?it/s]
  0%|          | 0/96 [00:00<?, ?it/s]


File: 2006_world_series_of_poker_results_edit_scores.csv
Thread: Allen Cunningham link


  0%|          | 0/96 [00:00<?, ?it/s]

0/170 tokens changed back
Thread: Event 11 & 12 confusion
{'adds': Counter(), 'dels': Counter({'Paul': 1})}


  0%|          | 0/96 [00:21<?, ?it/s]

...

 70%|██████▉   | 67/96 [5:52:53<2:32:44, 316.02s/it]

39/3556 tokens changed back
File: piotr_iljicz_czajkowskij_edit_scores.csv


 70%|██████▉   | 67/96 [5:52:53<2:32:44, 316.03s/it]

Thread: "Meck" vs. "von Meck"
{'adds': Counter({'Tchaikovsky': 3, 'death': 2, 'one': 2, 'also': 2, 'would': 1, 'The': 1, 'After': 1, "Tchaikovsky's": 1, 'death,': 1, 'score': 1, 'St': 1, 'He': 1, 'wrote': 1, 'Taneyev': 1, 'Op.': 1, 'known': 1, 'minor,': 1, 'Eugene': 1, 'although': 1}), 'dels': Counter()}
{'adds': Counter({'music': 2, 'also': 2, '1890.': 1}), 'dels': Counter({'symphony': 1, 'although': 1, 'Rubinstein': 1})}
{'adds': Counter({'The': 1}), 'dels': Counter({'No.': 2, 'This': 1})}
{'adds': Counter({'Taneyev': 1}), 'dels': Counter()}
{'adds': Counter({'Tchaikovsky': 3, 'never': 2, 'although': 1}), 'dels': Counter()}
{'adds': Counter({'A': 2, 'would': 1, 'The': 1}), 'dels': Counter()}
{'adds': Counter({'also': 2, "Tchaikovsky's": 1, 'He': 1, 'brother': 1}), 'dels': Counter({'one': 1})}
{'adds': Counter({'Russian': 3}), 'dels': Counter({'would': 3, 'generally': 1})}
{'adds': Counter({"Tchaikovsky's": 1, 'later': 1}), 'dels': Counter({'Tchaikovsky': 11, 'von': 7, "composer's": 3

KeyboardInterrupt: 

In [5]:
sum(double_count_percentage)/len(double_count_percentage)

0.004407834094967042

In [7]:
sum(double_count_percentage)/len(double_count_percentage)

0.015370502303974687

In [7]:
c = Counter(['a', 'a', 'a'])
c

Counter({'a': 3})

In [8]:
d = Counter(['a'])
c-d

Counter({'a': 2})

In [9]:
d-c

Counter()

# Print NLTK stopwords

In [10]:
stops = nltk.corpus.stopwords.words('english')
with open('/home/michael/Desktop/stopwords_en.txt', 'w') as f:
    for s in stops:
        f.write(s + '\n')

# Get all article names from talk data

In [2]:
talkpath = '/home/michael/school/research/wp/wikipedia/data/talk/enwiki/talk_filtered_2.csv'
talk = pd.read_csv(talkpath)
talk

,contribution_id,article,thread,timestamp,username,thread_type,text
0,1,"""Tiger"" William Dunlop",Requested move,2010-07-21 17:13:40,Billinghurst,THREAD_STARTER,The following discussion is an archived discus...
1,2,"""Tiger"" William Dunlop",Requested move,2010-07-14 10:47:31,CJ3370,POST,"William ""Tiger"" Dunlop → William ""Tiger"" Dunlo..."
2,3,"""Tiger"" William Dunlop",Requested move,2010-07-14 19:49:58,Skinsmoke,POST,User:CJ3370/William_Tiger_Dunlop → William Tig...
3,4,"""Tiger"" William Dunlop",Requested move,2010-07-14 10:47:31,CJ3370,POST,I'd like to make this contribution to Tiger Du...
4,5,"""Tiger"" William Dunlop",Requested move,2010-07-14 15:12:03,Skinsmoke,POST,Oppose This will result in two articles on th...
5,6,"""Tiger"" William Dunlop",Requested move,2010-07-14 16:59:04,Deconstructhis,POST,Not a problem to merge if that is the most app...
6,7,"""Tiger"" William Dunlop",Requested move,2010-07-14 19:49:58,Skinsmoke,POST,"Comment Sorry that you took offense, but it w..."
7,8,"""Tiger"" William Dunlop",Requested move,2010-07-21 17:13:40,Billinghurst,POST,The above discussion is preserved as an archiv...
8,9,"""Tiger"" William Dunlop",Current image,2010-07-15 20:19:17,Deconstructhis,THREAD_STARTER,I've noticed that the current image that's acc...
9,10,"""Tiger"" William Dunlop",Current image,2010-07-15 20:30:15,Ukexpat,POST,I removed the deletion tag because the image i...


In [3]:
artnames = sorted(set(talk['article']))
len(artnames)

11217

In [4]:
artnames[:10]

['"Tiger" William Dunlop',
 '"V" Is for Vagina',
 '&',
 "'Abd al-Ilah",
 "'Ndrangheta",
 "'Til Death Do Us Party",
 '(278361) 2007 JJ43',
 "(I Can't Get No) Satisfaction",
 '-ana',
 '.442 Webley']

In [5]:
outpath = '/home/michael/school/research/wp/talk_pagenames_100k_filtered2.txt'
with open(outpath, 'w') as out:
    for name in artnames:
        out.write(name + '\n')

In [5]:
# Transpose editor_citations
data = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/editor_citation_matrix.csv', index_col=0)
data = data.transpose()
data

,ABC,Al Jazeera,Al-Ahram,Amnesty International,Anti-Defamation League,Arutz Sheva (Israel News),Asia Times,Australian Broadcasting Company,B'Tselem,BBC,...,United Press International,Voice of America,Wall Street Journal,Washington Institute,Washington Post,Washington Times,Xinhuanet News,Yahoo News,Yale University,Ynet
%=),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
(14$H,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-The Bold Guy-,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-jmac-,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
.anacondabot,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
01011000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
01horng,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
069952497a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0DavidBerg0,0,0,0,0,0,0,0,0,-1,0,...,0,0,0,0,0,0,0,0,0,0
0kmck4gmja,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
data.shape

(14814, 100)

In [7]:
data.to_csv('/home/michael/school/research/wp/wikipedia/data/editor_citation_matrix.csv')

In [8]:
# Compare ipc article lists
fp2015 = '/home/michael/school/research/wp/wikipedia/data/ipc_articles.tsv'
fp2014 = '/home/michael/school/research/wp/wikipedia/data/lsipc.txt'

ipclist2015 = []
ipclist2014_norm = []

with open(fp2015, 'r') as blfile:
    reader = csv.reader(blfile)
    ipclist2015 = [row[0] for row in reader]
    
with open(fp2014, 'r') as artfile:
    ipclist2014_norm = artfile.read().splitlines()
    
print(len(ipclist2015))
print(len(ipclist2014_norm))

2040
2011


In [15]:
ipclist2014 = [n for n in ipclist2015 if n.lower().replace(' ', '_').replace('/', '_')+'.tsv' in ipclist2014_norm]
len(ipclist2014)

2011

In [18]:
# Write out IPC list 2014
t = pd.Series(ipclist2014)
t.to_csv('/home/michael/school/research/wp/wikipedia/data/ipc_articles_2014.tsv', index=False)

In [13]:
ipclist2014

['101 Squadron (Israel)',
 '16th Summit of the Non-Aligned Movement',
 '1920 Nebi Musa riots',
 '1929 Hebron massacre',
 '1929 Palestine riots',
 '1936–39 Arab revolt in Palestine',
 '1945 Anti-Jewish Riots in Tripolitania',
 '1947 Aden riots',
 '1947 Aleppo pogrom',
 '1947 Manama riots',
 '1947–48 Civil War in Mandatory Palestine',
 '1948 Anti-Jewish Riots in Oujda and Jerada',
 '1948 Arab–Israeli War',
 '1948 Cairo bombings',
 '1948 Palestine war',
 '1948 Palestinian exodus',
 '1948 Palestinian exodus from Lydda and Ramle',
 '1949 Armistice Agreements',
 '1949–56 Palestinian exodus',
 "1953 Iranian coup d'état",
 "1963 Syrian coup d'état",
 "1966 Syrian coup d'état",
 '1967 Arab League summit',
 '1967 Palestinian exodus',
 '1968 Polish political crisis',
 '1970 Arab League summit',
 '1974 Arab League summit',
 "1974 Beit She'an attack",
 '1974 Nahariya attack',
 '1976 Arab League summit (Cairo)',
 '1976 Arab League summit (Riyadh)',
 '1978 Arab League summit',
 '1978 South Lebanon co

In [31]:
nq = 4
modelpath = '/home/michael/school/cprose_research/wp/wikipedia/data/editor_categories_{:d}q.csv'.format(nq)
model = pd.read_csv(modelpath)

modelmatches = []
for row in model.values:
    if row[4] == row[5]:
        modelmatches.append(1)
    elif isinstance(row[4], float) and isinstance(row[5], float):
        if np.isnan(row[4]) and np.isnan(row[5]):
            modelmatches.append(1)
    else:
        modelmatches.append(0)
        
print(len(modelmatches))

print(modelmatches.count(1))

wikipath = '/home/michael/school/cprose_research/wp/wikipedia/data/editor_categories_wiki_{:d}q.csv'.format(nq)
wiki = pd.read_csv(wikipath)

wikimatches = []
for row in wiki.values:
    if row[4] == row[5]:
        wikimatches.append(1)
    elif isinstance(row[4], float) and isinstance(row[5], float):
        if np.isnan(row[4]) and np.isnan(row[5]):
            wikimatches.append(1)
    else:
        wikimatches.append(0)
        
print(len(wikimatches))

print(wikimatches.count(1))

ttest_ind(modelmatches, wikimatches)

1650
812
1787
904


(array(-0.8055753712274253), 0.42054349122742141)

In [ ]:
# Count accuracy, correlation
ncorrect = 0

for row in data.values:
    a = row[-2]
    b = row[-1]
    if row[-2] == row[-1]:
        ncorrect += 1
    elif isinstance(a, float) and isinstance(b, float):
        if np.isnan(row[-2]) and np.isnan(row[-1]):
            ncorrect += 1

print('Accuracy', ncorrect, '/', len(data), ncorrect/len(data))
print('Correlation', pearsonr(data['citation_bias'].values, data['sum_bias'].values))